<a href="https://colab.research.google.com/github/jason96819/Studying/blob/main/rnn/RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RNN을 이용한 텍스트 생성
'경마장에 있는 말이 뛰고 있다'
'그의 말이 법이다'
'가는 말이 고와야 오는 말이 곱다'
세가지 문장을 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려해 학습하도록 데이터를 재구성하겠습니다.

## 데이터에 대한 이해와 전처리

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# 3개의 한국어 문장 저장

text = """ 경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
# 단어 집합의 크기를 저장할 떄는 케라스 토크나이저이 정수 인코딩은 인덱스가 1부터 시작하지만,
# 패딩을 위한 0을 고려해 +1을 해줍니다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' %vocab_size)

단어 집합의 크기 : 12


In [ ]:
# 각 단어와 단어에 부여된 정수 인덱스를 출력해보겠습니다.

print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [ ]:
# 훈련 데이터를 만들어보겠습니다.

sequences = list()
for line in text.split('\n'):                                                                 # 줄바꿈 문자를 기준으로 문장 토큰화
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수 : %d' % len(sequences))

학습에 사용할 샘플의 개수 : 11


In [ ]:
# 전체 샘플을 출력해보겠습니다.

print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [ ]:
'''
위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다.
[2, 3]은 [경마장에, 있는]에 해당되며, [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다.
전체 훈련 데이터에 대해 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해 길이를 일치시켜줍니다.
가장 긴 샘플의 길이를 기준으로 합니다. 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다.
이를 코드로는 다음과 같이 구할 수 있습니다.
'''

'\n위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다.\n[2, 3]은 [경마장에, 있는]에 해당되며, [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다.\n전체 훈련 데이터에 대해 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.\n\n우선 전체 샘플에 대해 길이를 일치시켜줍니다.\n가장 긴 샘플의 길이를 기준으로 합니다. 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다.\n이를 코드로는 다음과 같이 구할 수 있습니다.\n'

In [ ]:
max_len = max(len(l) for l in sequences)         # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력

print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [ ]:
# 전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인했습니다. 전체 샘플의 길이를 6으로 패딩하겠습니다.

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

'''
pad_sequences()는 모든 샘플에 대해 0을 사용해 길이를 맞춰줍니다.
maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며,
padding의 인자로 'pre'로 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다.
전체 훈련 데이터를 출력해 보겠습니다.
'''

print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [ ]:
'''
길이가 6보다 짧은 모든 샘플에 대해 앞에 0을 채워 모든 샘플의 길이를 6으로 바꿨습니다.
이제 각 샘플의 마지막 단어를 레이블로 분리하겠습니다.
레이블의 분리는 numpy를 이용해 리스트의 마지막 값을 제외하고 저장한 것은 X, 리스트의 마지막 값만 저장한 것은 Y.
이는 레이블에 해당됩니다.
'''

sequences = np.array(sequences)
X = sequences[:, :-1]
Y = sequences[:, -1]

print(X)
print(Y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [ ]:
# 레이블이 분리되었습니다. RNN 모델에 훈련 데이터를 훈련시키기 전에 레이블에 대해 원-핫 인코딩을 수행합니다.

Y= to_categorical(Y, num_classes=vocab_size)

print(Y)                                                              # 각각 3에대한 원-핫 벡터, 1에 대한 원-핫 벡터, 4, 5, 1, 7, 1, 9, 10, 1, 11

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
'''
임베딩 벡터의 차원은 10
은닉 상태의 크기는 32
다 대 일 구조의 RNN을 사용합니다.
전결합층을 출력층으로 단어 집합 크기만큼의 뉴런을 배치해 모델을 설계합니다.
해당 모델은 마지막시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다.
다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수를 사용하고,
손실 함수로 크로스 엔트로피 함수를 사용해 200에포크 수행합니다.
'''

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, Y, epochs=200, verbose=2)

In [ ]:
# 모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어 출력해보겠습니다.

def sentence_generation(model, toeknizer, current_word, n):  #모델, 토크나이저, 현재 단어, 반복할 횟수
  init_word = current_word
  sentence = ' '

  # n번 반복
  for _ in range(n):
    # 현재 단어에 대한 정수 인코딩과 패딩
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')

    # 입력한 X(현재 단어)에 대해 Y를 예측하고 Y(예측한 단어)를 result에 저장
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for word, index in tokenizer.word_index.items():
      # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
      if index == result:
        break

    # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    current_word = current_word + ' ' + word

    # 예측 단어를 문장에 저장
    sentence = sentence + ' ' + word

  sentence = init_word + sentence
  return sentence

In [ ]:
'''
입력된 단어로부터 다음 단어를 예측해 문장을 생성하는 함수를 만들었습니다.
'경마장에'라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측해보겠습니다.
'''

print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에  있는 말이 뛰고 있다


In [ ]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는  말이 고와야 오는 말이 곱다


In [ ]:
'''
앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줍니다.
이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는
횟수 4, 2, 5를 각각 인자값으로 주었습니다. 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에
나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 합니다. 이번에는 더 많은 훈련 데이터를 가지고 실습해봅시다.
'''

# LSTM을 이용한 텍스트 생성
이번에는 LSTM을 통해 보다 많은 데이터로 텍스트를 생성해보겠습니다.

## 데이터에 대한 이해와 전처리
사용할 데이터는 뉴욕 타입즈 기사의 제목입니다.

In [ ]:
import pandas as pd
import numpy as np

from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# 다운로드한 훈련 데이터를 데이터프레임에 저장합니다.

df = pd.read_csv("ArticlesApril2018.csv")
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [ ]:
# 총 15개의 열이 존재합니다. 여기서 사용할 열은 제목에 해당되는 headline 열입니다.

print('열의 개수 :',len(df.columns))
print(df.columns)

열의 개수 : 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [ ]:
# null 값이 있는지 확인

print(df['headline'].isnull().values.any())

False


In [ ]:
# headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.

headline = []

headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [ ]:
# Unknown값을 가진 샘플이있습니다. 비록 null은 아니지만 도움이 되지 않는 노이즈 데이터이므로 제거해줄 필요가있습니다.
# 제거하기전 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해보겠습니다.

print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


In [ ]:
# 노이즈 데이터를 제거하기 전 신문 기사의 제목 샘플은 총 1,324개 입니다. Unkown값을 가진 샘플을 제거하겠습니다.

headline = [word for word in headline if word != "Unknown"]

print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [ ]:
# 기존의 출력했던 5개의 샘플을 출력해보겠습니다.

headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [ ]:
# 이제 데이터 전처리를 수행합니다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다.
# 전처리를 수행하고 다시 샘플 5개를 출력해보겠습니다.

def repreprocessing(raw_sentence):
  preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii", 'ignore')
  # 구두점 제거와 동시에 소문자화
  return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preproceseed_headline = [repreprocessing(x) for x in headline]
preproceseed_headline[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift its still looking',
 'romney failed to win at utah convention but few believe hes doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now hes meeting with pope francis']

In [ ]:
# 단어 집합(vocabulary)을 만들고 크기를 확인합니다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preproceseed_headline)
vocab_size = len(tokenizer.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [ ]:
# 정수 인코딩을 진행하는 동시에 하나의 문장을 여러 줄로 분해해 훈련 데이터를 구성합니다.

sequences = list()

for sentence in preproceseed_headline:

  # 각 샘플에 대한 정수 인코딩
  encoded = tokenizer.texts_to_sequences([sentence])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [ ]:
# 위의 sequences는 모든 문장을 각 단어가 각 시점마다 하나씩 추갖거으로 등장하는 형태로 만들기는 했지만,
# 아직 예측할 단어에 해당되는 레이블을 분리하는 작업까지는 수행하지 않았습니다.
# 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만들겠습니다.

index_to_word = {}
for key, value in tokenizer.word_index.items():    # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
  index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [ ]:
# 이제 y데이터를 분리하기전에 전체 샘플의 길이를 동일하게 만드는 패딩 작업을 수행합니다.
# 패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인합니다.

max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [ ]:
# 가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩하겠습니다.

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115  582]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582   52]]


In [ ]:
# 이제 맨 우측 단어만 레이블로 분리하겠습니다.

sequences = np.array(sequences)
X = sequences[:, :-1]
Y = sequences[:, -1]

print(X[:3])
print(Y)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115 ...  115    2 1025]


In [ ]:
# 레이블 데이터 Y에 대해 원-핫 인코딩을 수행하겠습니다.

Y = to_categorical(Y, num_classes=vocab_size)

## 모델 설계하기
하이퍼파라미터인 임베딩 벡터의 차원은 10,
은닉 상태의 크기는 128입니다.
다 대 일 구조의 LSTM을 사용합니다.
전결합층(Fully Connected Layer)을 출력층으로 단어 집합 크기만큼의 뉴런을 배치하여 모델을 설계합니다.
해당 모델은 마지막 시점에서 모든 가능한 단어 중 하나의 단어를 예측하는 다중 클래스 분류 문제를 수행하는 모델입니다.
다중 클래스 분류 문제의 경우, 출력층에 소프트맥스 회귀를 사용해야 하므로 활성화 함수로는 소프트맥스 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용하여 200 에포크를 수행합니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, Y, epochs=200)

Epoch 1/200
244/244 [==============================] - 8s 6ms/step - loss: 7.6475 - accuracy: 0.0309
Epoch 2/200
244/244 [==============================] - 1s 6ms/step - loss: 7.1061 - accuracy: 0.0309
Epoch 3/200
244/244 [==============================] - 1s 6ms/step - loss: 6.9674 - accuracy: 0.0349
Epoch 4/200
244/244 [==============================] - 1s 6ms/step - loss: 6.8353 - accuracy: 0.0414
Epoch 5/200
244/244 [==============================] - 1s 6ms/step - loss: 6.6802 - accuracy: 0.0452
Epoch 6/200
244/244 [==============================] - 1s 6ms/step - loss: 6.5039 - accuracy: 0.0496
Epoch 7/200
244/244 [==============================] - 1s 6ms/step - loss: 6.3113 - accuracy: 0.0552
Epoch 8/200
244/244 [==============================] - 1s 6ms/step - loss: 6.1164 - accuracy: 0.0578
Epoch 9/200
244/244 [==============================] - 1s 6ms/step - loss: 5.9263 - accuracy: 0.0618
Epoch 10/200
244/244 [==============================] - 1s 6ms/step - loss: 5.7494 - accura

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model, tokenizer, 'i', 10))


i disapprove of school vouchers can i still apply for them


In [ ]:
print(sentence_generation(model, tokenizer, 'what', 15))


what the heck is that niobe wont be alive on the past of died and weapons
